# Draft Model

In [1]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import requests

In [2]:
ADP_URL = "https://www.fantasypros.com/nfl/adp/ppr-overall.php"

In [3]:
def make_adp_df():
    res = requests.get(ADP_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id':'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n',df.head(),'\n')
        df = df[['Player Team (Bye)','POS','AVG']]
        print('Output after filtering:\n\n',df.head(),'\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x:' '.join(x.split()[:-2]))
        df['POS'] = df['POS'].apply(lambda x:x[:2])
        
        df = df[['PLAYER','POS','AVG']].sort_values(by='AVG')
        
        print('Final output:\n\n', df.head())
        
        return df
    
    else:
        print("Didn't work, try again")

In [4]:
df = make_adp_df()

Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      4.0  3.3
3     4         Michael Thomas NO (6)  WR1   6.0  NaN       5.0      3.0  4.7
4     5           Dalvin Cook MIN (7)  RB4   4.0  NaN       4.0      6.0  4.7 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.3
3         Michael Thomas NO (6)  WR1  4.7
4           Dalvin Cook MIN (7)  RB4  4.7 

Final output:

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2      Ezekiel Elliott  RB  3.3
3       Michael Thomas  WR  4.7
4          Dalvin Cook  RB  4.7


In [5]:
df.head()

,PLAYER,POS,AVG
0,Christian McCaffrey,RB,1.0
1,Saquon Barkley,RB,2.0
2,Ezekiel Elliott,RB,3.3
3,Michael Thomas,WR,4.7
4,Dalvin Cook,RB,4.7


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596 entries, 0 to 595
Data columns (total 3 columns):
PLAYER    596 non-null object
POS       596 non-null object
AVG       596 non-null float64
dtypes: float64(1), object(2)
memory usage: 18.6+ KB


In [7]:
replacement_players = {
    'RB':None,
    'WR':None,
    'TE':None,
    'QB':None
}

In [8]:
replacement_players

{'RB': None, 'WR': None, 'TE': None, 'QB': None}

In [9]:
# look at the top 100 picks in our df and get the last RB, WR, QB, and TE
# and store them in our dict

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player
    
replacement_players

{'RB': 'Kerryon Johnson',
 'WR': 'Sterling Shepard',
 'TE': 'Jared Cook',
 'QB': 'Carson Wentz'}

Get projected player scores

In [10]:
BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

def make_projection_df():
    
    final_df = pd.DataFrame()
    
    for position in ['rb', 'qb', 'te', 'wr']:
        
        res = requests.get(BASE_URL.format(position=position))
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0)
            df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1]))
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC']
                
            df['POS'] = position.upper()
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df])
            
        else:
            print("Didn't work, try again")
            return
    
    final_df = final_df.sort_values(by='FPTS', ascending=False)
    
    return final_df

In [11]:
df = make_projection_df()
df.head()

,PLAYER,POS,FPTS
0,Christian McCaffrey,RB,375.0
0,Lamar Jackson,QB,361.1
1,Patrick Mahomes,QB,345.0
0,Michael Thomas,WR,333.1
2,Dak Prescott,QB,318.4


Calculate replacement values

In [12]:
replacement_values = {
    'RB':None,
    'WR':None,
    'TE':None,
    'QB':None
}

replacement_values

{'RB': None, 'WR': None, 'TE': None, 'QB': None}

In [13]:
for position, player in replacement_players.items():
    if position in ['QB','TE', 'WR', 'RB']:
        replacement_values[position] = df.loc[df['PLAYER'] == player].values[0,-1]
        
replacement_values

{'RB': 149.6, 'WR': 183.8, 'TE': 146.2, 'QB': 274.5}

In [14]:
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), 
    axis=1
)
df.head()

,PLAYER,POS,FPTS,VOR
0,Christian McCaffrey,RB,375.0,225.4
0,Lamar Jackson,QB,361.1,86.6
1,Patrick Mahomes,QB,345.0,70.5
0,Michael Thomas,WR,333.1,149.3
2,Dak Prescott,QB,318.4,43.9


In [15]:
df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)
df.head(50)

,PLAYER,POS,FPTS,VOR,VALUERANK
0,Christian McCaffrey,RB,375.0,225.4,1.0
1,Saquon Barkley,RB,310.8,161.2,2.0
2,Ezekiel Elliott,RB,299.6,150.0,3.0
0,Michael Thomas,WR,333.1,149.3,4.0
5,Alvin Kamara,RB,296.2,146.6,5.0
4,Dalvin Cook,RB,289.9,140.3,6.0
1,Davante Adams,WR,298.8,115.0,7.0
3,Derrick Henry,RB,261.4,111.8,8.0
0,Travis Kelce,TE,252.7,106.5,9.0
6,Clyde Edwards-Helaire,RB,254.1,104.5,10.0


In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
# fig = plt.figure(figsize=(10,8))

# sns.lineplot(x='VALUERANK', y='VOR', data=df[df['POS'] == 'RB'])

# plt.show()

In [18]:
positions = list(set(df.POS.values))
positions

['QB', 'RB', 'WR', 'TE']

In [19]:
# for p in positions:
#     sns.lineplot(x='VALUERANK', y='VOR', data=df[df['POS'] == p])
#     plt.title(p)
#     plt.show()

# Avg Draft Pick

In [21]:
adp_df = make_adp_df()
adp_df.head()

Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      4.0  3.3
3     4         Michael Thomas NO (6)  WR1   6.0  NaN       5.0      3.0  4.7
4     5           Dalvin Cook MIN (7)  RB4   4.0  NaN       4.0      6.0  4.7 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.3
3         Michael Thomas NO (6)  WR1  4.7
4           Dalvin Cook MIN (7)  RB4  4.7 

Final output:

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2      Ezekiel Elliott  RB  3.3
3       Michael Thomas  WR  4.7
4          Dalvin Cook  RB  4.7


,PLAYER,POS,AVG
0,Christian McCaffrey,RB,1.0
1,Saquon Barkley,RB,2.0
2,Ezekiel Elliott,RB,3.3
3,Michael Thomas,WR,4.7
4,Dalvin Cook,RB,4.7


In [22]:
adp_df['ADPRANK'] = adp_df['AVG'].rank(method='first')

adp_df.head()

,PLAYER,POS,AVG,ADPRANK
0,Christian McCaffrey,RB,1.0,1.0
1,Saquon Barkley,RB,2.0,2.0
2,Ezekiel Elliott,RB,3.3,3.0
3,Michael Thomas,WR,4.7,4.0
4,Dalvin Cook,RB,4.7,5.0


Merge dataframes

In [23]:
df = df.merge(adp_df, how='left', on=['PLAYER', 'POS'])

df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.6,150.0,3.0,3.3,3.0
3,Michael Thomas,WR,333.1,149.3,4.0,4.7,4.0
4,Alvin Kamara,RB,296.2,146.6,5.0,5.3,6.0


In [24]:
df.head(100)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.6,150.0,3.0,3.3,3.0
3,Michael Thomas,WR,333.1,149.3,4.0,4.7,4.0
4,Alvin Kamara,RB,296.2,146.6,5.0,5.3,6.0
5,Dalvin Cook,RB,289.9,140.3,6.0,4.7,5.0
6,Davante Adams,WR,298.8,115.0,7.0,11.3,8.0
7,Derrick Henry,RB,261.4,111.8,8.0,7.3,7.0
8,Travis Kelce,TE,252.7,106.5,9.0,18.0,20.0
9,Clyde Edwards-Helaire,RB,254.1,104.5,10.0,16.0,17.0


In [25]:
df['SLEEPERSCORE'] = df['ADPRANK'] - df['VALUERANK']

df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0,0.0
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0
2,Ezekiel Elliott,RB,299.6,150.0,3.0,3.3,3.0,0.0
3,Michael Thomas,WR,333.1,149.3,4.0,4.7,4.0,0.0
4,Alvin Kamara,RB,296.2,146.6,5.0,5.3,6.0,1.0


# Sleeper Score 

In [26]:
df.loc[df['AVG'] < 160].sort_values(by='SLEEPERSCORE', ascending=False).head(20)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
112,Jonnu Smith,TE,130.1,-16.1,113.0,159.3,160.0,47.0
117,Jack Doyle,TE,126.6,-19.6,118.0,156.3,156.0,38.0
107,Dallas Goedert,TE,132.3,-13.9,108.0,146.7,145.0,37.0
90,Mike Gesicki,TE,146.0,-0.2,91.0,130.0,126.0,35.0
113,Ryan Tannehill,QB,258.2,-16.3,114.0,147.7,146.0,32.0
57,Josh Allen,QB,297.4,22.9,58.0,90.0,89.0,31.0
115,DeSean Jackson,WR,165.4,-18.4,116.0,148.7,147.0,31.0
106,Jared Goff,QB,260.7,-13.8,107.0,140.0,136.0,29.0
83,Hayden Hurst,TE,148.2,2.0,84.0,113.7,112.0,28.0
61,Tarik Cohen,RB,171.0,21.4,62.0,88.7,87.0,25.0


In [27]:
df.loc[df['AVG'] < 160].sort_values(by='SLEEPERSCORE', ascending=True).head(20)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
337,Ke'Shawn Vaughn,RB,21.9,-127.7,338.0,138.7,135.0,-203.0
176,Alexander Mattison,RB,94.5,-55.1,177.5,127.3,122.0,-55.5
187,Mecole Hardman,WR,123.0,-60.8,188.0,136.0,133.0,-55.0
203,Alshon Jeffery,WR,106.3,-77.5,204.0,154.5,153.0,-51.0
137,J.K. Dobbins,RB,119.1,-30.5,138.0,90.7,90.0,-48.0
91,Leonard Fournette,RB,147.9,-1.7,92.5,51.7,50.0,-42.5
155,Darrell Henderson,RB,109.3,-40.3,156.0,124.0,119.0,-37.0
153,Justin Jefferson,WR,143.9,-39.9,154.0,119.3,117.0,-37.0
139,Latavius Murray,RB,118.8,-30.8,140.0,107.3,103.0,-37.0
144,Jerry Jeudy,WR,147.9,-35.9,145.0,114.7,113.0,-32.0


# Keeper Analysis

In [28]:
keeper_df = pd.read_excel('Keeper_Master Document.xlsx',sheet_name=None)

In [29]:
keeper_df.keys()

odict_keys(['Keeper History', '2020', '2019', '2018', '2017', '2016 ', '2015'])

In [30]:
keeper_df['Keeper History'].head()

,Year,Team,Player,Round,Position,Notes
0,2015,David,Adrian Peterson,1,NaN,NaN
1,2015,David,Randall Cobb,3,NaN,NaN
2,2015,David,Alshon Jeffrey,13,NaN,NaN
3,2015,Dinatalie,Eddie Lacy,4,NaN,NaN
4,2015,Dinatalie,Jordan Matthews,17,NaN,NaN


In [31]:
keeper_history = keeper_df['Keeper History']
keeper_history.head()

,Year,Team,Player,Round,Position,Notes
0,2015,David,Adrian Peterson,1,NaN,NaN
1,2015,David,Randall Cobb,3,NaN,NaN
2,2015,David,Alshon Jeffrey,13,NaN,NaN
3,2015,Dinatalie,Eddie Lacy,4,NaN,NaN
4,2015,Dinatalie,Jordan Matthews,17,NaN,NaN


In [32]:
keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [33]:
keepers_2020

array(['Davante Adams', 'George Kittle', 'DJ Moore', 'DJ Chark',
       'Juju Smith Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'DK Metcalf',
       'Brashad Perrimen', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'AJ Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliot ',
       'Nick Chubb', 'Devante Parker ', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a '], dtype=object)

In [34]:
len(df.PLAYER.values)

607

# Name Replacements

In [200]:
name_replacements = [('DJ Moore', 'D.J. Moore'),
 ('DJ Chark', 'D.J. Chark'),
 ('Juju Smith Schuster', 'JuJu Smith-Schuster'),
 ('JuJu Smith', 'JuJu Smith-Schuster'),
 ('DK Metcalf', 'D.K. Metcalf'),
 ('Brashad Perrimen', 'Breshad Perriman'),
 ('AJ Brown', 'A.J. Brown'),
 ('Ronald Jones', 'Ronald Jones II'),
 ('Ezekiel Elliot ', 'Ezekiel Elliott'),
 ('Ezekiel Elliot', 'Ezekiel Elliott'),
 ('Devante Parker ', 'DeVante Parker'),
 ('Devante Parker', 'DeVante Parker'),
 ('Saquon Barkely', 'Saquon Barkley'),
 ('Christian McAffrey', 'Christian McCaffrey'),
 ('Adam Thielan', 'Adam Thielen'),
 ('LeVeon Bell', "Le'Veon Bell"),
 ('Leornard Fournette', 'Leonard Fournette'),
 ('Mark Ingram', 'Mark Ingram II'),
 ('Mark Ingram ', 'Mark Ingram II'),
 ('TY Hilton', 'T.Y. Hilton'),
 ('Derius Guice', 'Derrius Guice'),
 ('AJ Green', 'A.J. Green'),
 ('Odell Beckham', 'Odell Beckham Jr.'),
 ('L McCoy', 'LeSean McCoy'),
 ('Alshon Jeffrey', 'Alshon Jeffery'),
 ('Marvin Jones Jr', 'Marvin Jones'),
 ('Darell Henderson', 'Darrell Henderson'),
 ('Russel Wilson', 'Russell Wilson'),
 ('A Miller', 'Anthony Miller'),
 ('DJ Metcalf', 'D.K. Metcalf'),
 ('Desean Jackson', 'DeSean Jackson'),
 ('Gio Bernard', 'Giovani Bernard'),
 ('Justice Mill', 'Justice Hill'),
 ('Kennan Ballage', 'Kalen Ballage'),
 ('Malcom Brown', 'Malcolm Brown'),
 ('Marques Valdez Scandling', 'Marquez Valdes-Scantling'),
 ('OJ Howard', 'O.J. Howard'),
 ('Philip Lindsay', 'Phillip Lindsay'),
 ('R Higgins', 'Rashard Higgins'),
 ('Rashad Penny', 'Rashaad Penny'),
 ('Tyreke Hill', 'Tyreek Hill'),
 ('TJ Hockenson', 'T.J. Hockenson')]

In [195]:
kickers = ['Greg Zuerlin',
 'Harrison Butker',
 'Justin Tucker',
 'Kai Fairbarn',
 'Mike Badgley',
 'Robbie Gould',
 'Stephen Gostowski',
 'Will Lutz',
 'Younghoe Koo',
 'Wil Lutz',
 'Austin Seibert']

In [170]:
defenses = ['Bears',
 'Rams',
 'Browns',
 'Eagles',
 'Jaguars',
 'Patriots',
 'Ravens',
 'Vikings',
 '49ers',
 'Broncos',
 'Chiefs',
 'Falcons',
 'Jets',
 'Pats']

In [180]:
unsigned_fa = ['Devonta Freeman',
 'Antonio Brown',
 'Derrius Guice',
 'Damien Williams',
 'Delanie Walker',
 'Geronimo Allison',
 'Josh Gordon',
 'Taylor Gabriel',
 'Tyrell Williams',
 'Adrian Peterson']

## Cleaning Keeper List

In [106]:
def check_names(check_list, correct_list):
    l = []
    for p in check_list:
        if p not in correct_list:
            l.append(p)
    return l

def find_name(name, source=df.PLAYER.values):
    for p in source:
        if name in p:
            print(p)
            
def name_replacer(wrong_name, correct_name, data=draft_19):
    data.replace(to_replace=wrong_name, value=correct_name, inplace=True)
    
def remove_fa_k_dst(main_list, fa, k, dst):
    main_list = np.setdiff1d(main_list, fa)
    main_list = np.setdiff1d(main_list, k)
    main_list = np.setdiff1d(main_list, dst)
    return main_list

In [40]:
keeper_replacements = [
    ('DJ Moore', 'D.J. Moore'), 
    ('DJ Chark', 'D.J. Chark'),
    ('Juju Smith Schuster', 'JuJu Smith-Schuster'),
    ('DK Metcalf', 'D.K. Metcalf'),
    ('Brashad Perrimen', 'Breshad Perriman'),
    ('AJ Brown', 'A.J. Brown'),
    ('Ronald Jones', 'Ronald Jones II'),
    ('Ezekiel Elliot ', 'Ezekiel Elliott'),
    ('Devante Parker ', 'DeVante Parker')
]

In [43]:
for p in keeper_replacements:
    keeper_history.replace(to_replace=p[0], value=p[1], inplace=True)

keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
    
keepers_2020

array(['Davante Adams', 'George Kittle', 'D.J. Moore', 'D.J. Chark',
       'JuJu Smith-Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'D.K. Metcalf',
       'Breshad Perriman', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'A.J. Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones II', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliott',
       'Nick Chubb', 'DeVante Parker', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a '], dtype=object)

In [44]:
names_to_fix = check_names(keepers_2020, df.PLAYER.values)
names_to_fix

['Adrian Peterson', 'N/a ', 'N/a ']

In [45]:
# keeper_history.replace(to_replace='DJ Moore', value='D.J. Moore', inplace=True)

In [42]:
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [48]:
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [49]:
# for p in df.PLAYER.values:
#     if 'Chark' in p:
#         print(p)

In [45]:
# keeper_history.replace(to_replace='DJ Chark', value='D.J. Chark', inplace=True)

In [50]:
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [51]:
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [52]:
# for p in df.PLAYER.values:
#     if 'Schuster' in p:
#         print(p)

In [49]:
# keeper_history.replace(to_replace='Juju Smith Schuster', value='JuJu Smith-Schuster', inplace=True)

In [50]:
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values

In [53]:
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [54]:
# for p in df.PLAYER.values:
#     if 'Metcalf' in p:
#         print(p)

In [55]:
# keeper_history.replace(to_replace='DK Metcalf', value='D.K. Metcalf', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [54]:
# for p in df.PLAYER.values:
#     if 'Perriman' in p:
#         print(p)

Breshad Perriman


In [56]:
# keeper_history.replace(to_replace='Brashad Perrimen', value='Breshad Perriman', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [57]:
# for p in df.PLAYER.values:
#     if 'Brown' in p:
#         print(p)

In [58]:
# keeper_history.replace(to_replace='AJ Brown', value='A.J. Brown', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [59]:
# for p in df.PLAYER.values:
#     if 'Jones' in p:
#         print(p)

In [60]:
# keeper_history.replace(to_replace='Ronald Jones', value='Ronald Jones II', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [61]:
# for p in df.PLAYER.values:
#     if 'Ezekiel' in p:
#         print(p)

In [62]:
# keeper_history.replace(to_replace='Ezekiel Elliot ', value='Ezekiel Elliott', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

In [62]:
# for p in df.PLAYER.values:
#     if 'Parker' in p:
#         print(p)

DeVante Parker


In [63]:
# keeper_history.replace(to_replace='Devante Parker ', value='DeVante Parker', inplace=True)
# keepers_2020 = keeper_history[keeper_history['Year'] == 2020].Player.dropna().values
# names_to_fix = check_names(keepers_2020, df.PLAYER.values)
# names_to_fix

['Adrian Peterson', 'N/a ', 'N/a ']

In [63]:
# keepers cleaned
keepers_2020

array(['Davante Adams', 'George Kittle', 'D.J. Moore', 'D.J. Chark',
       'JuJu Smith-Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'D.K. Metcalf',
       'Breshad Perriman', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'A.J. Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones II', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliott',
       'Nick Chubb', 'DeVante Parker', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a '], dtype=object)

# Last Year's Draft Picks

In [64]:
draft_19 = keeper_df['2019']

In [65]:
draft_19.columns[:8]

Index(['Tony', 'Mike', 'Joey', 'Vlad', 'Dave', 'Chris', 'Matt', 'Ben'], dtype='object')

In [66]:
draft_19.index[:18]

Index(['Round 1', 'Round 2', 'Round 3', 'Round 4', 'Round 5', 'Round 6',
       'Round 7', 'Round 8', 'Round 9', 'Round 10', 'Round 11', 'Round 12',
       'Round 13', 'Round 14', 'Round 15', 'Round 16', 'Round 17', 'Round 18'],
      dtype='object')

In [68]:
draft_19 = draft_19.iloc[:, :8]
draft_19

,Tony,Mike,Joey,Vlad,Dave,Chris,Matt,Ben
Round 1,Saquon Barkely,Ezekiel Elliot,Deshaun Watson,Julio Jones,Todd Gurley,Travis Kelce,Antonio Brown,Christian McAffrey
Round 2,Adam Thielan,Josh Jacobs,Amari Cooper,Chris Carson,Zach Ertz,Dalvin Cook,LeVeon Bell,Keenan Allen
Round 3,David Montgomery,Sony Michel,Leornard Fournette,Mike Williams,Mark Ingram,Melvin Gordon,Derrick Henry,Kenny Golladay
Round 4,Brandin Cooks,Kerryon Johnson,Stefon Diggs,Chris Godwin,DeAndre Hopkins,Julian Edelman,TY Hilton,Derius Guice
Round 5,Tevin Coleman,Mike Evans,Austin Ekeler,Evan Engram,Tyler Boyd,Calvin Ridley,AJ Green,Tyler Lockett
Round 6,Allen Robinson,Robert Woods,Tony Pollard,Devin Singletary,Christian Kirk,Odell Beckham,L McCoy,Miles Sanders
Round 7,Aaron Rodgers,Robby Anderson,Jarvis Landry,Baker Mayfield,Duke Johnson,Alshon Jeffrey,Courtland Sutton,Cam Newton
Round 8,Jordan Howard,Marvin Jones Jr,Sammy Watkins,Darell Henderson,Russel Wilson,Dede Westbrook,Kenyan Drake,Latavius Murray
Round 9,Matt Breida,Vance McDonald,James White,Larry Fitzgerald,Tarik Cohen,Greg Zuerlin,Emmanuel Sanders,Jared Cook
Round 10,Devonta Freeman,Curtis Samuel,Geronimo Allison,Rashad Penny,Justin Tucker,Mike Thomas,James Washington,Kyler Murray


In [72]:
all_players_19 = draft_19.values.flatten()

In [74]:
all_players_19

array(['Saquon Barkely', 'Ezekiel Elliot', 'Deshaun Watson',
       'Julio Jones', 'Todd Gurley', 'Travis Kelce', 'Antonio Brown',
       'Christian McAffrey', 'Adam Thielan', 'Josh Jacobs',
       'Amari Cooper', 'Chris Carson', 'Zach Ertz', 'Dalvin Cook',
       'LeVeon Bell', 'Keenan Allen', 'David Montgomery', 'Sony Michel',
       'Leornard Fournette', 'Mike Williams', 'Mark Ingram',
       'Melvin Gordon', 'Derrick Henry', 'Kenny Golladay',
       'Brandin Cooks', 'Kerryon Johnson', 'Stefon Diggs', 'Chris Godwin',
       'DeAndre Hopkins', 'Julian Edelman', 'TY Hilton', 'Derius Guice',
       'Tevin Coleman', 'Mike Evans', 'Austin Ekeler', 'Evan Engram',
       'Tyler Boyd', 'Calvin Ridley', 'AJ Green', 'Tyler Lockett',
       'Allen Robinson', 'Robert Woods', 'Tony Pollard',
       'Devin Singletary', 'Christian Kirk', 'Odell Beckham', 'L McCoy',
       'Miles Sanders', 'Aaron Rodgers', 'Robby Anderson',
       'Jarvis Landry', 'Baker Mayfield', 'Duke Johnson',
       'Alshon Je

In [75]:
names_to_fix = check_names(all_players_19, df.PLAYER.values)

In [83]:
len(names_to_fix)

127

In [84]:
# original_keepers = [p[0] for p in keeper_replacements]

# for p in original_keepers:
#     if p in names_to_fix:
#         print(p)

In [201]:
for p in name_replacements:
    draft_19.replace(to_replace=p[0], value=p[1], inplace=True)

all_players_19 = draft_19.values.flatten()
    
names_to_fix = check_names(all_players_19, df.PLAYER.values)
len(names_to_fix)

91

In [202]:
names_to_fix = remove_fa_k_dst(names_to_fix, unsigned_fa, kickers, defenses)
names_to_fix

array(['nan'], dtype='<U17')

In [203]:
names_to_fix

array(['nan'], dtype='<U17')

In [204]:
draft_19

,Tony,Mike,Joey,Vlad,Dave,Chris,Matt,Ben
Round 1,Saquon Barkley,Ezekiel Elliott,Deshaun Watson,Julio Jones,Todd Gurley,Travis Kelce,Antonio Brown,Christian McCaffrey
Round 2,Adam Thielen,Josh Jacobs,Amari Cooper,Chris Carson,Zach Ertz,Dalvin Cook,Le'Veon Bell,Keenan Allen
Round 3,David Montgomery,Sony Michel,Leonard Fournette,Mike Williams,Mark Ingram II,Melvin Gordon,Derrick Henry,Kenny Golladay
Round 4,Brandin Cooks,Kerryon Johnson,Stefon Diggs,Chris Godwin,DeAndre Hopkins,Julian Edelman,T.Y. Hilton,Derrius Guice
Round 5,Tevin Coleman,Mike Evans,Austin Ekeler,Evan Engram,Tyler Boyd,Calvin Ridley,A.J. Green,Tyler Lockett
Round 6,Allen Robinson,Robert Woods,Tony Pollard,Devin Singletary,Christian Kirk,Odell Beckham Jr.,LeSean McCoy,Miles Sanders
Round 7,Aaron Rodgers,Robby Anderson,Jarvis Landry,Baker Mayfield,Duke Johnson,Alshon Jeffery,Courtland Sutton,Cam Newton
Round 8,Jordan Howard,Marvin Jones,Sammy Watkins,Darrell Henderson,Russell Wilson,Dede Westbrook,Kenyan Drake,Latavius Murray
Round 9,Matt Breida,Vance McDonald,James White,Larry Fitzgerald,Tarik Cohen,Greg Zuerlin,Emmanuel Sanders,Jared Cook
Round 10,Devonta Freeman,Curtis Samuel,Geronimo Allison,Rashaad Penny,Justin Tucker,Mike Thomas,James Washington,Kyler Murray


# Keep Filtering

In [205]:
df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0,0.0
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0
2,Ezekiel Elliott,RB,299.6,150.0,3.0,3.3,3.0,0.0
3,Michael Thomas,WR,333.1,149.3,4.0,4.7,4.0,0.0
4,Alvin Kamara,RB,296.2,146.6,5.0,5.3,6.0,1.0


In [206]:
keepers_2020

array(['Davante Adams', 'George Kittle', 'D.J. Moore', 'D.J. Chark',
       'JuJu Smith-Schuster', 'Lamar Jackson', 'Kenyan Drake',
       'Raheem Mostert', 'Austin Ekeler', 'D.K. Metcalf',
       'Breshad Perriman', 'Adrian Peterson', 'Mark Andrews',
       'Terry McLaurin', 'A.J. Brown', 'James Conner', 'Tyreek Hill',
       'Alvin Kamara', 'Ronald Jones II', 'Tyler Boyd', 'Derrick Henry',
       'Chris Godwin', 'Joe Mixon', 'Aaron Jones', 'Ezekiel Elliott',
       'Nick Chubb', 'DeVante Parker', 'N/a ', 'Dalvin Cook',
       'Michael Thomas', 'Patrick Mahomes', 'N/a '], dtype=object)

In [207]:
df['KEEPER'] = df.PLAYER.isin(keepers_2020)
df.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0,0.0,False
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0,False
2,Ezekiel Elliott,RB,299.6,150.0,3.0,3.3,3.0,0.0,True
3,Michael Thomas,WR,333.1,149.3,4.0,4.7,4.0,0.0,True
4,Alvin Kamara,RB,296.2,146.6,5.0,5.3,6.0,1.0,True


## Begin df_no_keep

In [208]:
df_no_keep = df[df["KEEPER"] == False].copy()
df_no_keep

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0,0.0,False
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0,False
8,Travis Kelce,TE,252.7,106.5,9.0,18.0,20.0,11.0,False
9,Clyde Edwards-Helaire,RB,254.1,104.5,10.0,16.0,17.0,7.0,False
11,Julio Jones,WR,286.4,102.6,12.0,16.3,18.0,6.0,False
12,Miles Sanders,RB,249.4,99.8,13.0,13.3,10.0,-3.0,False
18,DeAndre Hopkins,WR,261.2,77.4,19.0,12.0,9.0,-10.0,False
21,Josh Jacobs,RB,222.6,73.0,22.0,14.3,13.0,-9.0,False
24,Todd Gurley,RB,216.5,66.9,25.0,30.7,29.0,4.0,False
26,Zach Ertz,TE,211.1,64.9,27.0,37.7,36.0,9.0,False


## Add Keep Pick Eligibility

In [218]:
keep_el = pd.read_excel('Keeper_Eligibility.xlsx', sheet_name=None)

In [232]:
keep_el['penalty_keeps']

,PLAYER,keep_eligibilty
0,Christian McCaffrey,Unkeepable
1,Todd Gurley,Unkeepable
2,Kerryon Johnson,3
3,Mike Evans,1
4,Devonta Freeman,6
5,Davante Adams,11
6,Phillip Lindsay,15
7,George Kittle,16
8,Nick Chubb,10
9,D.J. Moore,16


In [220]:
for p in name_replacements:
    keep_el['penalty_keeps'].replace(to_replace=p[0], value=p[1], inplace=True)

In [222]:
for p in name_replacements:
    keep_el['maintain_position'].replace(to_replace=p[0], value=p[1], inplace=True)

In [225]:
df_no_keep.drop(columns=['KEEP_PICK_ELIGIBILITY'], inplace=True)
df_no_keep.head()

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER
0,Christian McCaffrey,RB,375.0,225.4,1.0,1.0,1.0,0.0,False
1,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0,False
8,Travis Kelce,TE,252.7,106.5,9.0,18.0,20.0,11.0,False
9,Clyde Edwards-Helaire,RB,254.1,104.5,10.0,16.0,17.0,7.0,False
11,Julio Jones,WR,286.4,102.6,12.0,16.3,18.0,6.0,False


In [236]:
merge1 = df_no_keep.merge(keep_el['penalty_keeps'],how='left', on='PLAYER')

In [237]:
merge2 = merge1.merge(keep_el['maintain_position'], how='inner', on='PLAYER')

In [241]:
pd.set_option('display.max_rows', None)

In [243]:
merge2.drop(columns=['keep_eligibilty_x'], inplace=True)

In [244]:
df_final = merge2.copy()

In [245]:
df_final['VALUE_ROUND'] = (df_final['VALUERANK'] // 10) + 1 
df_final

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,keep_eligibilty_y,VALUE_ROUND
0,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0,False,1,1.0
1,Travis Kelce,TE,252.7,106.5,9.0,18.0,20.0,11.0,False,1,1.0
2,Julio Jones,WR,286.4,102.6,12.0,16.3,18.0,6.0,False,1,2.0
3,Miles Sanders,RB,249.4,99.8,13.0,13.3,10.0,-3.0,False,6,2.0
4,DeAndre Hopkins,WR,261.2,77.4,19.0,12.0,9.0,-10.0,False,1,2.0
5,Josh Jacobs,RB,222.6,73.0,22.0,14.3,13.0,-9.0,False,1,3.0
6,Todd Gurley,RB,216.5,66.9,25.0,30.7,29.0,4.0,False,1,3.0
7,Zach Ertz,TE,211.1,64.9,27.0,37.7,36.0,9.0,False,1,3.0
8,Le'Veon Bell,RB,212.4,62.8,28.0,37.0,35.0,7.0,False,1,3.0
9,Chris Carson,RB,212.2,62.6,29.0,32.3,31.0,2.0,False,1,3.0


In [246]:
df_final['keep_score'] = df_final['keep_eligibilty_y'] - df_final['VALUE_ROUND']
df_final.head(10)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,keep_eligibilty_y,VALUE_ROUND,keep_score
0,Saquon Barkley,RB,310.8,161.2,2.0,2.0,2.0,0.0,False,1,1.0,0.0
1,Travis Kelce,TE,252.7,106.5,9.0,18.0,20.0,11.0,False,1,1.0,0.0
2,Julio Jones,WR,286.4,102.6,12.0,16.3,18.0,6.0,False,1,2.0,-1.0
3,Miles Sanders,RB,249.4,99.8,13.0,13.3,10.0,-3.0,False,6,2.0,4.0
4,DeAndre Hopkins,WR,261.2,77.4,19.0,12.0,9.0,-10.0,False,1,2.0,-1.0
5,Josh Jacobs,RB,222.6,73.0,22.0,14.3,13.0,-9.0,False,1,3.0,-2.0
6,Todd Gurley,RB,216.5,66.9,25.0,30.7,29.0,4.0,False,1,3.0,-2.0
7,Zach Ertz,TE,211.1,64.9,27.0,37.7,36.0,9.0,False,1,3.0,-2.0
8,Le'Veon Bell,RB,212.4,62.8,28.0,37.0,35.0,7.0,False,1,3.0,-2.0
9,Chris Carson,RB,212.2,62.6,29.0,32.3,31.0,2.0,False,1,3.0,-2.0


# Final Analysis



In [253]:
df_final.sort_values(by='SLEEPERSCORE', ascending=False)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE,KEEPER,keep_eligibilty_y,VALUE_ROUND,keep_score
72,Gerald Everett,TE,84.7,-61.5,189.0,265.0,300.0,111.0,False,18,19.0,-1.0
74,Josh Gordon,WR,112.3,-71.5,197.0,258.3,291.0,94.0,False,18,20.0,-2.0
86,Mike Boone,RB,27.3,-122.3,315.0,330.0,406.0,91.0,False,18,32.0,-14.0
68,Jerick McKinnon,RB,98.0,-51.6,173.0,230.0,246.0,73.0,False,18,18.0,0.0
76,Will Dissly,TE,65.3,-80.9,208.0,251.0,277.0,69.0,False,18,21.0,-3.0
84,Zach Pascal,WR,67.8,-116.0,291.0,295.0,352.0,61.0,False,18,30.0,-12.0
77,Dion Lewis,RB,63.7,-85.9,220.0,234.0,252.0,32.0,False,18,23.0,-5.0
53,Ryan Tannehill,QB,258.2,-16.3,114.0,147.7,146.0,32.0,False,18,12.0,6.0
70,O.J. Howard,TE,89.9,-56.3,181.0,206.3,212.0,31.0,False,18,19.0,-1.0
55,DeSean Jackson,WR,165.4,-18.4,116.0,148.7,147.0,31.0,False,13,12.0,1.0


In [255]:
df_final.to_csv('position_analysis.csv')